<a href="https://colab.research.google.com/github/SaikrishnaParamkusham/NLP_GEN_AI/blob/master/Assignment_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load the sentiment analysis data (test.csv) Data: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset?select=test.csv

2. You have to perform some preprocessing

3. There should be two columns (text, sentiment)

4. Then perform text encoding or embedding over the text data and label encoding on target column

5. Write a RNN code in PyTorch from scratch to train sentiment analysis model on batch data using Dataset and Dataloader

6. Then test it out on random example

In [38]:
# Loading all the packages
import pandas as pd
import numpy as np
import kagglehub
import torch

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [37]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
# Install kaggle
!pip install kaggle

# Upload your kaggle.json file (API key)
from google.colab import files
files.upload()

# Make a directory named kaggle and copy kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d abhi8923shriv/sentiment-analysis-dataset

# Unzip the dataset
!unzip sentiment-analysis-dataset.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset
License(s): CC0-1.0
 97% 53.0M/54.4M [00:03<00:00, 27.6MB/s]
100% 54.4M/54.4M [00:03<00:00, 14.8MB/s]
Archive:  sentiment-analysis-dataset.zip
  inflating: test.csv                
  inflating: testdata.manual.2009.06.14.csv  
  inflating: train.csv               
  inflating: training.1600000.processed.noemoticon.csv  


In [5]:
# Assuming 'test.csv' is the correct filename after unzipping.
# Adjust the path if necessary.
try:
  df_test = pd.read_csv('test.csv', encoding='latin-1')
except FileNotFoundError:
  print("Error: 'test.csv' not found. Check the dataset extraction.")

In [6]:
try:
  df_train = pd.read_csv('train.csv', encoding='latin-1')
except FileNotFoundError:
  print("Error: 'test.csv' not found. Check the dataset extraction.")

In [7]:
df_test.head(1)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0


In [8]:
df_test.shape

(4815, 9)

In [9]:
df_train.shape

(27481, 10)

In [10]:
df_train.columns

Index(['textID', 'text', 'selected_text', 'sentiment', 'Time of Tweet',
       'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)',
       'Density (P/Km²)'],
      dtype='object')

Data Understanding

In [11]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [12]:
df_train = df_train[['text', 'sentiment']]

In [15]:
df_train.columns

Index(['text', 'sentiment'], dtype='object')

In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [17]:
df_train['sentiment'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [18]:
# Identifies dominant categories.
# Useful for data imbalance detection.
df_train['sentiment'].value_counts()

,count
sentiment,
neutral,11118
positive,8582
negative,7781


In [19]:
# Checking for the missing values
df_train.isnull().sum()

,0
text,1
sentiment,0


Text Preprocessing

In [21]:
# Removing Missing values
df_train.dropna(inplace=True)

In [22]:
df_train.isnull().sum()

,0
text,0
sentiment,0


Preprocessing the natural launage (text column)

In [23]:
df_train['text'][2]

'my boss is bullying me...'

# Text Cleaning
1.   Remove Unwanted Characters
2.   Lowercasing
3.   Remove numbers

# Tokenization
1.   Covert text into small units, tokens (words or subwords)

# Stop Word
1.   Remove stop words such as "the", "is", "in" and others

# Stemming
1.   Reduce the word to root form. For example, "running", "runner" to "run"

# Lemmatization
1.   Reducing to root word with a proper meaning.

# Removing Rare and Common Words
1.   Rare - Word that appear too infrequently in the dataset.
2.   Common - Word that appear too frequently in the dataset.

# Handling Misspelling
1.   Spell correcction

# Part-of-Speech Tagging
1.   Helps to understand the role of a word in a sentence.

# Named Entity Recognition (NER)
1.   NER identifies proper nouns such as names of people, places, and organizations.

# Vectorization (Feature Extraction)
In order for the machine learning model to understand text, the words need to be represented as numerical data:

1.   Bag of Words (BoW): Represents text as a vector based on the count of words.
2.   TF-IDF (Term Frequency - Inverse Document Frequency): Adjusts word counts based on how common a word is across all documents.
3.   Word Embeddings: Represent words as dense vectors (e.g., using Word2Vec, GloVe, or FastText).

# Text Representation (Advanced Techniques)
1.   Word2Vec: Uses neural networks to convert words into vector representations.
2.   GloVe: Pre-trained word embeddings that represent words in a continuous vector space.

In [39]:
def text_Cleaning(df, column, new_column):
    """
    Cleans text data in the specified column of a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame
    column (str): The column name to clean

    Returns:
    pd.DataFrame: DataFrame with cleaned text

    """
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    def preprocess(text):

      if pd.isna(text):
        return ""

      text = text.lower()
      text = re.sub(r"[^\w\s]", "", text) # Removes punctuation
      text = re.sub(r"\d+", "", text)     # Remove numbers

      tokens = word_tokenize(text)
      tokens = [word for word in tokens if word not in stop_words]
      tokens = [lemmatizer.lemmatize(word) for word in tokens]

      return " ".join(tokens)

    df[new_column] = df[column].astype(str).apply(preprocess)
    return df

In [40]:
df = text_Cleaning(df_train, "text", "cleaned_text")

In [43]:
df["cleaned_text"][1]

'sooo sad miss san diego'

In [44]:
df["text"][1]

' Sooo SAD I will miss you here in San Diego!!!'

In [47]:
df["text"][0]

' I`d have responded, if I were going'

In [45]:
df["cleaned_text"][0]

'id responded going'

In [48]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from textblob import TextBlob
import spacy

# Download necessary resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("omw-1.4")

# Load SpaCy model for Named Entity Recognition (NER)
nlp = spacy.load("en_core_web_sm")

def clean_text(df, column, new_column, rare_threshold=2, common_threshold=0.5):
    """
    Cleans text data in the specified column and stores it in a new column.

    Parameters:
    df (pd.DataFrame): The input DataFrame
    column (str): The column name containing raw text
    new_column (str): The column name to store cleaned text
    rare_threshold (int): The minimum frequency for rare words (default: 2)
    common_threshold (float): The fraction threshold for common words (default: 50%)

    Returns:
    pd.DataFrame: DataFrame with the new cleaned text column
    """

    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Compute word frequencies
    all_words = [word for text in df[column].dropna() for word in word_tokenize(str(text).lower())]
    word_freq = Counter(all_words)

    # Define rare and common words
    rare_words = {word for word, freq in word_freq.items() if freq <= rare_threshold}
    common_words = {word for word, freq in word_freq.items() if freq >= common_threshold * len(df)}

    def preprocess(text):
        if pd.isna(text):  # Handle missing values
            return ""

        # Convert to lowercase
        text = text.lower()

        # Remove punctuation and unwanted characters
        text = re.sub(r"[^\w\s]", "", text)

        # Remove numbers
        text = re.sub(r"\d+", "", text)

        # Tokenization
        tokens = word_tokenize(text)

        # Stopword Removal
        tokens = [word for word in tokens if word not in stop_words]

        # Stemming
        stemmed_tokens = [stemmer.stem(word) for word in tokens]

        # Lemmatization
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in stemmed_tokens]

        # Remove Rare and Common Words
        tokens = [word for word in lemmatized_tokens if word not in rare_words and word not in common_words]

        # Spell Correction
        corrected_tokens = [TextBlob(word).correct() for word in tokens]

        # POS Tagging
        pos_tags = nltk.pos_tag(corrected_tokens)

        # Named Entity Recognition (NER)
        doc = nlp(" ".join(corrected_tokens))
        named_entities = [(ent.text, ent.label_) for ent in doc.ents]

        return {
            "cleaned_text": " ".join(corrected_tokens),
            "pos_tags": pos_tags,
            "named_entities": named_entities
        }

    # Apply preprocessing and extract data
    processed_data = df[column].astype(str).apply(preprocess)

    # Store cleaned text, POS tags, and NER results in new columns
    df[new_column] = processed_data.apply(lambda x: x["cleaned_text"])
    df["pos_tags"] = processed_data.apply(lambda x: x["pos_tags"])
    df["named_entities"] = processed_data.apply(lambda x: x["named_entities"])

    return df


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
